In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from ATLAS_style import *
ROOT.gErrorIgnoreLevel = ROOT.kError
set_atlas_style()

In [ ]:
file_HIST_ref = ROOT.TFile.Open("release_68/out.HIST.root")
file_HIST_new = ROOT.TFile.Open("release_72/out.HIST.root")

In [ ]:
base_dir = 'run_470000/HLT/TauMon/HLT_Efficiency'
file_base_dir = file_HIST_ref.Get(base_dir)
HLTpath_dir = []

keys = file_base_dir.GetListOfKeys()
for key in keys:
    obj = key.ReadObj()
    if obj.InheritsFrom("TDirectory"):
        HLTpath_dir.append(obj.GetName())

HLTpath_subdir = ['HLT_Efficiency_1P', 'HLT_Efficiency_3P']
HLTpath_eff = ['EffHLT_averageMu_wrt_Offline',
               'EffHLT_tauEta_highPt_wrt_Offline', 'EffHLT_tauEta_wrt_Offline',
               'EffHLT_tauPhi_highPt_wrt_Offline', 'EffHLT_tauPhi_wrt_Offline',
               'EffHLT_tauPt_coarse_wrt_Offline', 'EffHLT_tauPt_wrt_Offline',
              ]

In [ ]:
def compare_eff(path, eff_ref, eff_new, release_ref, release_new, ymin=0, ymax=1.6):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    eff_ref.SetLineColor(ROOT.kBlack)
    eff_new.SetLineColor(ROOT.kRed)
    
    eff_ref.Draw("AP")
    eff_new.Draw("P SAME")

    canvas.Update()

    graph_ref = eff_ref.GetPaintedGraph()
    if graph_ref:
        graph_ref.GetYaxis().SetRangeUser(ymin, ymax)

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/HLT_Efficiency/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_Efficiency_3P/EffHLT_tauPt_wrt_Offline'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '1P' in path_parts[5]:
        prong = '1P'
    elif '3P' in path_parts[5]:
        prong = '3P'
    
    legend = ROOT.TLegend(0.2, 0.87, 0.45, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + prong + ' ' + path_parts[6])
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(20)
    legend.SetTextSize(20)
    legend.AddEntry(eff_ref, release_ref, "lp")
    legend.AddEntry(eff_new, release_new, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.2, 0.88, "Internal")

    save_dir = 'plots/' + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for HLT_path in HLTpath_dir:
    for prong in HLTpath_subdir:
        for eff in HLTpath_eff:
            eff_path = base_dir + '/' + HLT_path + '/' + prong + '/' + eff
            compare_eff(eff_path,
                        file_HIST_ref.Get(eff_path),
                        file_HIST_new.Get(eff_path),
                        '24.0.68',
                        '24.0.72',
                        ymin=0,
                        ymax=1.6
                       )